# Libraries

In [1]:
import jupyternotify
import requests
import pandas as pd
import calendar
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from datetime import datetime
from datetime import timedelta
from time import sleep
import json
import os

ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
pd.options.display.float_format = '{:,.2f}'.format

<IPython.core.display.Javascript object>

# API Params

Header de la API

In [2]:
header_sales = {
    'Content-Type': 'application/json',
    'Authorization': 'ISVToken 682a99604c5a4cb4ab4cf222090ef79e'
}

URL para distinguir qué vamos a descargar

In [3]:
url_sales = 'https://api.instoreview.cl/api/v2/download-zone/sales/'

El body que indica las columnas cualitativas que ocuparemos

In [4]:
body_sales = {
    'view_type':'week',
    'dates': [],
    'views': ['Unidades', 'Precio Lista'],
    'hierarchy':{
        'Cadena': [],
        'Sub Cadena': [],
        'Suc. ID': [],
        'EAN': [],
        'Descripción Producto': [],
        'Pro.Pst.ID': [],
        'Cad. ID':[]
    }
}

# Functiones

Función para limpiar las columnas cuantitativas

In [5]:
def clean_numbers(num):
    try:
        num + 0
    except:
        num = float(str(num).replace('.','').replace(',',''))/1000000
    return num

Función para descomprimir el archivo zip y leer el csv

In [6]:
def unpack_file(response):
    try:
        file = urlopen(response.json()['download_url'])
        zip_file = ZipFile(BytesIO(file.read()))
        df = pd.read_csv(zip_file.open(zip_file.namelist()[0]), encoding='latin-1', sep=',')
    except:
        df = pd.DataFrame()
    return df

Función para validar si la descarga fue correcta

In [7]:
def check_status(response, date):
    status = response.status_code
    if status != 200:
        print("Problems in '{}':".format(date), '\n', 'Error {0}: {1}'.format(status, resp_sales.text))

Función para descargar el sell out desde ISV

In [8]:
def download_sales(year, num_week, url, header, body):
    body["dates"] = ['{0}-W{1}'.format(year, str(num_week).zfill(2))]
    resp_sales = requests.post(url, data=json.dumps(body), headers=header)
    check_status(resp_sales, body_sales["dates"][0])
    df = unpack_file(resp_sales)
    return df

Función para limpiar las columnas de *views*

In [9]:
def clean_views(data, views):
    for col in views:
        data[col] = data[col].map(clean_numbers).copy()
    return data

Función para limpiar las columnas de *hierarchy*

In [10]:
def clean_hierarchy(data, hierarchy):
    import numpy as np
    for col in hierarchy:
        data.loc[data[col]=='No Definido', col] = np.nan
    data['EAN'] = data['EAN'].astype(str)
    return data

Función que integra ambas funciones (clean_views y clean_hierarchy)

In [11]:
def clean_sales(df, views, hierarchy):
    data = df.copy()
    df_views = clean_views(data[views], views)
    df_hierarchy = clean_hierarchy(data[hierarchy], hierarchy)
    return pd.concat([df_hierarchy, df_views], axis=1)

# Main

Los skus que vamos a descargar

In [ ]:
ean_to_check = [
    650240020803, 
    650240024306, 
    650240026591, 
    650240026607, 
    650240026614, 
    7501199407036, 
    650240053993, 
    650240054020, 
    650240054037, 
    650240054068
]

Agregamos en el body los EAN y el cliente

In [13]:
body_sales['hierarchy']['EAN'] = ean_to_check

body_sales['hierarchy']['Cadena'] = ['Soriana']

Generamos una lista de tuplas por año, semana que consultaremos

In [14]:
year_week = [(2022, w) for w in range(20, 24)]

Descarga de datos de ISV

In [15]:
%%time
df = pd.DataFrame()
for year, week in year_week:
    aux = download_sales(year, week, url=url_sales, body=body_sales, header=header_sales)
    df = pd.concat([df, aux], axis=0)
    # Detemos la consulta 20 segs
    sleep(20)
del aux

CPU times: total: 266 ms
Wall time: 1min 46s


# Validate Data

In [16]:
df.shape

(115, 10)

In [17]:
df.dtypes

Semanas                  object
Cadena                   object
Sub Cadena               object
Suc. ID                   int64
Cad. ID                   int64
EAN                       int64
Descripción Producto     object
Pro.Pst.ID                int64
Unidades                float64
Precio Lista            float64
dtype: object

In [25]:
df.pivot_table(
    index=['Descripción Producto'],
    values=['Unidades'],
    columns=['Semanas'],
    aggfunc='sum'
)

Unidades  \
Semanas                                Semana 20 16-05-2022 a 22-05-2022   
Descripción Producto                                                       
Zanzusi Labial Lm Coral 03 1.6 grs                                  8.00   
Zanzusi Labial Lm Lila Neon 07 1.6 grs                              5.00   
Zanzusi Labial Lm Rojo 04 1.6 grs                                   6.00   
Zanzusi Labial Ss Beige 08 1.6 grs                                  6.00   

                                                                          \
Semanas                                Semana 21 23-05-2022 a 29-05-2022   
Descripción Producto                                                       
Zanzusi Labial Lm Coral 03 1.6 grs                                 15.00   
Zanzusi Labial Lm Lila Neon 07 1.6 grs                              3.00   
Zanzusi Labial Lm Rojo 04 1.6 grs                                   5.00   
Zanzusi Labial Ss Beige 08 1.6 grs                                 14.00   

                                                                          \
Semanas                                Semana 22 30-05-2022 a 05-06-2022   
Descripción Producto                                                       
Zanzusi Labial Lm Coral 03 1.6 grs                                 17.00   
Zanzusi Labial Lm Lila Neon 07 1.6 grs                              5.00   
Zanzusi Labial Lm Rojo 04 1.6 grs                                  17.00   
Zanzusi Labial Ss Beige 08 1.6 grs                                 18.00   

                                                                          
Semanas                                Semana 23 06-06-2022 a 12-06-2022  
Descripción Producto                                                      
Zanzusi Labial Lm Coral 03 1.6 grs                                  1.00  
Zanzusi Labial Lm Lila Neon 07 1.6 grs                               NaN  
Zanzusi Labial Lm Rojo 04 1.6 grs                                   1.00  
Zanzusi Labial Ss Beige 08 1.6 grs                                   NaN